# **Serialisation et Module FASTAPI **

In [4]:
!pip install FastAPI  uvicorn nest-asyncio pyngrok dill -q

In [37]:
import uvicorn
import joblib
import pickle
import dill
import lightgbm as lgb
from fastapi import FastAPI

from fastapi.middleware.cors import CORSMiddleware

from pydantic import BaseModel

class Client(BaseModel):

    AMT_ANNUITY: float

    EXT_SOURCES_MAX: float
def my_score(y_pred,y_true):
    tp = np.sum(y_pred * y_true, axis=0)
    fp = np.sum(y_pred * (1 - y_true), axis=0)
    fn = np.sum((1 - y_pred) * y_true, axis=0)
    cost = 15*fn + 5*fp #/ (2*tp + fn + fp + 1e-16)
    return cost

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

#with open("./model.dat", "rb") as f:

#    model = pickle.load(f)
model = lgb.Booster(model_file='model.txt')


@app.get('/')

def index():

    return {'message': 'This is the homepage of the API '}


@app.get('/client/{id}')

def get_client_id(id: int):

    
    return {'message': f'Hello! @{id}'}


@app.get('/prediction')

def get_model_decision(data: Client):

    received = data.dict()

    AMT_ANNUITY = received['AMT_ANNUITY']

    EXT_SOURCES_MAX = received['EXT_SOURCES_MAX']


    pred_name = model.predict([[AMT_ANNUITY, EXT_SOURCES_MAX]]).tolist()[0]

    return {'prediction': pred_name}



In [38]:
import nest_asyncio
nest_asyncio.apply()

In [39]:
if __name__ == '__main__':

    uvicorn.run(app, host='127.0.0.1', port=4000, debug=True)

INFO:     Started server process [14636]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57712 - "GET /client/10 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57715 - "GET /prediction HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14636]


In [40]:
from platform import python_version

print(python_version())

3.8.8
